# Lecture 23: Databases

![](https://www.tensorflow.org/images/colab_logo_32px.png)
[Run in colab](https://colab.research.google.com/drive/1l6-c2qrmddCsiyB9Avn1cS8v794pHBl-)

In [1]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

Last executed: 2022-03-07 01:30:31


As the volume of data we are handling grows, it becomes harder to keep track of how it is structured and what information it contains.

We need to store the data in a way that lets us inspect and update it in a straightforward, principled manner.

A **database** is a collection of data that offers easier and more efficient storage and retrieval.

Benefits of databases include:
- The ability to query the data to find entries of interest
- The ability to modify the contents
- A more abstract view of the data, focusing on high-level entities and relationships, rather than "low-level" variables
- Performance improvements through internal book-keeping
- Control over who can access their contents
 
Traditionally, the most common way of storing data like this has been in a relational database.

## Relational databases

In a relational database, data exists in tables:
- Rows correspond to individual records or data samples
- Columns are variables of a particular type


The structure of each table is critical. This includes:
- the number of columns
- the name of each column
- the type of data in each column

This is called the **schema** of the database. 

Deciding on the structure of the database is very important. A good schema aims to:
- capture all the relevant information
- avoid duplication of data as much as possible
- capture constraints about the data, such as types
- represent relations between different fields and tables

Here is an example (taken from a [Software Carpentry tutorial](https://swcarpentry.github.io/sql-novice-survey/01-select/index.html)) of how a schema can be depicted:

![Sample of a database schema, showing four tables, the columns in each table and their types, and the relationships between columns in different tables](https://swcarpentry.github.io/sql-novice-survey/fig/sql-join-structure.svg)

For each of the four tables, the schema lists its columns and their types. The arrows show dependencies between different tables. For example, the "person" column in the Survey table cannot contain arbitrary values; it must be a valid person ID, i.e. it must match the value of the "id" column for some row in the Person table. This is a common type of constraint that relational databases enforce. 

The database checks that these constraints and relations are satisfied at all times.

If the user tries to insert data that would violate the constraints, the database system will reject that update, ensuring the **integrity** of the data.

There are several **relational database management systems (RDBMSs)**, such as [MySQL](https://www.mysql.com/), [Postgresql](https://www.postgresql.org/), [SQLite](https://sqlite.org/) and [Oracle](https://oracle.com/database).

These offer slightly different features but share the same fundamental approach.

### SQL

To extract information from a relational database, we can write queries in the **Structured Query Language (SQL)**.

Common operations include:
- Selecting a subset of records based on the value of one or more fields
- Selecting only certain columns of interest
- Performing aggregations (summing or counting records)
- Combining  ("joining") rows from multiple tables


#### Example: Reading from an SQL database

To illustrate some of these concepts, we will work with a small database of fictional experimental measurements. You can read more about the structure of the database in the [tutorial](https://swcarpentry.github.io/sql-novice-survey/01-select/index.html) it originates from.

First we need to download the data. In this case, database itself is stored in a file which can be shared like any other file.

In [2]:
# Use Python's built-in support for sending and receiving data over the web
import urllib.request

# Get the database from a remote location...
response = urllib.request.urlopen("https://swcarpentry.github.io/sql-novice-survey/files/survey.db")
# ...and save it as a local file
with open('survey.db', 'wb') as db_file:
    contents = response.read()
    db_file.write(contents)

The database we have downloaded is for the SQLite system.

To issue SQL commands, we can either work with a RDBMS directly, or use a programming language and appropriate packages. Python comes with built-in support for SQLite databases through the [`sqlite3` package](https://docs.python.org/3/library/sqlite3.html). 

In [3]:
import sqlite3

connection = sqlite3.connect('survey.db')

Once connected, we can execute SQL queries and get the results.

#### Selecting rows of data

The fundamental way of extracting data with SQL is a query of the form:

```sql
SELECT <column(s) of interest>
FROM <corresponding table(s)>
```

For example, we can get the personal and family names (i.e. first and last name) of everyone in the database:

```sql
SELECT personal, family
FROM Person
```

Let's run this query in Python and print the results:

In [4]:
names = connection.execute("SELECT personal, family FROM Person")
for result in names:
    print(result)

('William', 'Dyer')
('Frank', 'Pabodie')
('Anderson', 'Lake')
('Valentina', 'Roerich')
('Frank', 'Danforth')


Usually we will want to get only a subset of the rows that satisfy a condition. We can restrict the results using an extra `WHERE` clause:

In [5]:
visits = connection.execute("SELECT * FROM Survey WHERE reading < 0")
for result in visits:
    print(result)

(734, 'pb', 'temp', -21.5)
(735, None, 'temp', -26.0)
(751, 'pb', 'temp', -18.5)
(752, 'lake', 'temp', -16.0)


Queries can look up and combine information in multiple tables.

For instance, even though the Survey table only holds scientists' ids, not their names, we can get all the measurements ("readings") made by a person if we know their last name:

```sql
SELECT reading
FROM Survey, Person
WHERE Survey.person = Person.id AND Person.family = 'Lake'
```

(This is an example - there are better ways of doing this kind of combination!)

When we run a query like this, the database system may not need to inspect every row to find the results.

Instead, it uses an internal set of **indices** that allow it to look up the relevant rows. These indices are updated automatically and stored alongside the tables.

This gives increased performance, without the user having to worry about maintaing these indices or even know the details of how queries run.

#### Modifying data

Apart from extracting data, SQL can be used to:
- insert new rows into a table
- modify existing rows
- delete rows

For example, the query

```sql
INSERT INTO Person
VALUES ('bta', 'Beatriz', 'Torres')
```

will create a new row in the Person table for a person named Beatriz Torres with id `bta`.

We can build up more complex queries, e.g. "increase by 10% every measurement which is below the mean, unless it was taken last week".

#### Other SQL features

SQL offers more functionality, such as:
- Basic arithmetic and computations
- Built-in functions for working with dates and other data types (depending on the RDBMS)
- Syntax for creating tables and altering their schema
- Syntax for specifying constraints and dependencies

More complex operations can be completed by "combining" SQL with a programming language.

In [6]:
# Terminate the connection when done
connection.close()

### Benefits of relational databases

Relational databases were practically the standard until recently. They offer:
- A shared way of using them (the core of SQL is the same despite variations in different RDBMSs)
- Guarantees about the integrity of the data
- Stability and a large community of experienced users
- Decades of theoretical research and performance optimisations

### Interacting with databases programmatically

There are different ways we can connect to a relational database, read and modify its contents. These include:
- Running the RDBMS application directly and issuing SQL commands from there
- For many programming languages, using a library to connect and run queries (like `sqlite3` in the examples above)
- Depending on the RDBMS, it may be possible to store a copy of the database into a file and then reload it
- Libraries like `pandas` also allow reading database tables and running queries 

In [7]:
import pandas as pd
# Read the table named Person from the SQLite database stored in the file survey.db
df = pd.read_sql_table("Person", "sqlite:///survey.db")
# The result is a normal DataFrame that we can manipulate as we wish
df

,id,personal,family
0,dyer,William,Dyer
1,pb,Frank,Pabodie
2,lake,Anderson,Lake
3,roe,Valentina,Roerich
4,danforth,Frank,Danforth


## Non-relational databases

Sometimes the rigid structure of a relational database is not useful. Non-relational or **NoSQL databases** can store data even when it does not have a consistent structure. This flexibility is useful, for example, when our data comes in various forms, or when we are still gathering it and therefore don't know what the best structure to use will be.

There are many NoSQL database systems, such as [MongoDB](https://www.mongodb.com/) and [Neo4j](https://neo4j.com/).

Instead of tables with fixed columns, these databases model their contents in different ways.

For example, MongoDB treats its contents as documents containing multiple fields. The fields don't need to be the same across documents: we can store entries with different types of information in the same database without issue. 

Instead of SQL, each system usually offers its own libraries for accessing a database through a programming language (for example, MongoDB has the [`pymongo`](https://pymongo.readthedocs.io/en/stable/) package for Python, and [similar for other languages](https://docs.mongodb.com/drivers/)).

Benefit: Access can be more intuitive, without needing to learn a new language.

Downside: Systems differ in how they expect users to structure and access their data.

### Comparison

The two approaches are most effective in different situations:

- SQL emphasises integrity and validation of data but requires a rigid structure; NoSQL offers flexibility but leaves checks up to the user.
- SQL structure focuses on eliminating duplication and improving performance; NoSQL is more concerned with availability and scaling, even if it means replicating data.
- SQL has an established presence and set of tools; NoSQL systems show more frequent innovation.

However, both sets of systems are now drawing inspiration from each other to offer new features.

### Summary

- Databases provide a principled way for storing, sharing and querying large amounts of data.
- Traditional relational databases can be queried through SQL.
- NoSQL databases offer more flexibility at the expense of integrity and consistency.